In [17]:
from keras.models import Model, load_model
from keras.layers import Input, Conv2D, GlobalAveragePooling2D, Dropout
from keras.layers import Activation, BatchNormalization, Add, Reshape, DepthwiseConv2D
from keras.utils.vis_utils import plot_model
from keras.layers.advanced_activations import LeakyReLU
from keras import backend as K
import numpy as np
from keras.utils import np_utils

In [4]:
np.random.seed(1230)

In [69]:

def _make_divisible(v, divisor, min_value=None):
    if min_value is None:
        min_value = divisor
    new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)
    # Make sure that round down does not go down by more than 10%.
    if new_v < 0.9 * v:
        new_v += divisor
    return new_v


def relu6(x):
    """Relu 6
    """
    return K.relu(x, max_value=6.0)


def _conv_block(inputs, filters, kernel, strides):
    """Convolution Block
    This function defines a 2D convolution operation with BN and relu6.
    # Arguments
        inputs: Tensor, input tensor of conv layer.
        filters: Integer, the dimensionality of the output space.
        kernel: An integer or tuple/list of 2 integers, specifying the
            width and height of the 2D convolution window.
        strides: An integer or tuple/list of 2 integers,
            specifying the strides of the convolution along the width and height.
            Can be a single integer to specify the same value for
            all spatial dimensions.
    # Returns
        Output tensor.
    """

    channel_axis = 1 if K.image_data_format() == 'channels_first' else -1

    x = Conv2D(filters, kernel, padding='same', strides=strides)(inputs)

    x = BatchNormalization(axis=channel_axis)(x)
    #x = BatchNormalizationF16(axis=channel_axis)(x)
    return LeakyReLU(alpha=0.03)(x)


def _bottleneck(inputs, filters, kernel, t, alpha, s, r=False):
    """Bottleneck
    This function defines a basic bottleneck structure.
    # Arguments
        inputs: Tensor, input tensor of conv layer.
        filters: Integer, the dimensionality of the output space.
        kernel: An integer or tuple/list of 2 integers, specifying the
            width and height of the 2D convolution window.
        t: Integer, expansion factor.
            t is always applied to the input size.
        s: An integer or tuple/list of 2 integers,specifying the strides
            of the convolution along the width and height.Can be a single
            integer to specify the same value for all spatial dimensions.
        alpha: Integer, width multiplier.
        r: Boolean, Whether to use the residuals.
    # Returns
        Output tensor.
    """

    channel_axis = 1 if K.image_data_format() == 'channels_first' else -1
    # Depth
    tchannel = K.int_shape(inputs)[channel_axis] * t
    # Width
    cchannel = int(filters * alpha)

    x = _conv_block(inputs, tchannel, (1, 1), (1, 1))

    x = DepthwiseConv2D(kernel, strides=(s, s), depth_multiplier=1, padding='same')(x)
    
    x = BatchNormalization(axis=channel_axis)(x)
    #x = BatchNormalizationF16(axis=channel_axis)(x)
    
    x = LeakyReLU(alpha=0.03)(x)

    x = Conv2D(cchannel, (1, 1), strides=(1, 1), padding='same')(x)
    
    x = BatchNormalization(axis=channel_axis)(x)
    #x = BatchNormalizationF16(axis=channel_axis)(x)
    
    if r:
        x = Add()([x, inputs])

    return x


def _inverted_residual_block(inputs, filters, kernel, t, alpha, strides, n):
    """Inverted Residual Block
    This function defines a sequence of 1 or more identical layers.
    # Arguments
        inputs: Tensor, input tensor of conv layer.
        filters: Integer, the dimensionality of the output space.
        kernel: An integer or tuple/list of 2 integers, specifying the
            width and height of the 2D convolution window.
        t: Integer, expansion factor.
            t is always applied to the input size.
        alpha: Integer, width multiplier.
        s: An integer or tuple/list of 2 integers,specifying the strides
            of the convolution along the width and height.Can be a single
            integer to specify the same value for all spatial dimensions.
        n: Integer, layer repeat times.
    # Returns
        Output tensor.
    """

    x = _bottleneck(inputs, filters, kernel, t, alpha, strides)

    for i in range(1, n):
        x = _bottleneck(x, filters, kernel, t, alpha, 1, True)

    return x


def MobileNetv2(input_shape, k, alpha=1.0):
    """MobileNetv2
    This function defines a MobileNetv2 architectures.
    # Arguments
        input_shape: An integer or tuple/list of 3 integers, shape
            of input tensor.
        k: Integer, number of classes.
        alpha: Integer, width multiplier, better in [0.35, 0.50, 0.75, 1.0, 1.3, 1.4].
    # Returns
        MobileNetv2 model.
    """
    inputs = Input(shape=input_shape)

    #first_filters = _make_divisible(32 * alpha, 8)
    first_filters = 16
    x = _conv_block(inputs, first_filters, (3, 3), strides=(2, 2))

    x = _inverted_residual_block(x, 16, (3, 3), t=1, alpha=alpha, strides=1, n=1)
    x = _inverted_residual_block(x, 32, (3, 3), t=1, alpha=alpha, strides=2, n=1)
    x = _inverted_residual_block(x, 64, (3, 3), t=1, alpha=alpha, strides=2, n=1)
    #x = _inverted_residual_block(x, 96, (3, 3), t=3, alpha=alpha, strides=1, n=2)
    x = _inverted_residual_block(x, 96, (3, 3), t=1, alpha=alpha, strides=2, n=1)
    #x = _inverted_residual_block(x, 320, (3, 3), t=6, alpha=alpha, strides=1, n=1)

    #if alpha > 1.0:
        #last_filters = _make_divisible(1280 * alpha, 8)
    #else:
        #last_filters = 1280
    last_filters = 32
    x = _conv_block(x, last_filters, (1, 1), strides=(1, 1))
    x = GlobalAveragePooling2D()(x)
    x = Reshape((1, 1, last_filters))(x)
    x = Dropout(0.3, name='Dropout')(x)
    x = Conv2D(k, (1, 1), padding='same')(x)

    x = Activation('softmax', name='softmax')(x)
    output = Reshape((k,))(x)

    model = Model(inputs, output)
    # plot_model(model, to_file='images/MobileNetv2.png', show_shapes=True)

    return model


if __name__ == '__main__':
    model = MobileNetv2((48, 48, 1), 7, 1.0)
    print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 48, 48, 1)         0         
_________________________________________________________________
conv2d_45 (Conv2D)           (None, 24, 24, 16)        160       
_________________________________________________________________
batch_normalization_57 (Batc (None, 24, 24, 16)        64        
_________________________________________________________________
leaky_re_lu_41 (LeakyReLU)   (None, 24, 24, 16)        0         
_________________________________________________________________
conv2d_46 (Conv2D)           (None, 24, 24, 16)        272       
_________________________________________________________________
batch_normalization_58 (Batc (None, 24, 24, 16)        64        
_________________________________________________________________
leaky_re_lu_42 (LeakyReLU)   (None, 24, 24, 16)        0         
__________

In [71]:
x = np.load("img_train.npy")
y = np.load("img_label.npy")

x_train = x[0:int(28709*0.8)]/255
y_train = y[0:int(28709*0.8)]
x_val = x[int(28709*0.8): 28709]/255
y_val = y[int(28709*0.8):28709]

In [72]:
model.compile(loss='categorical_crossentropy',
              optimizer= 'Adam',
              metrics=['accuracy'])
#print(MODEL.model.summary())

In [74]:
model.save_weights("test.h5")

In [75]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=25,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.3,
    zoom_range=[0.8, 1.2],
    horizontal_flip=True,
    vertical_flip=True
    )

datagen.fit(x_train)
train_generator = datagen.flow(x_train,y_train,batch_size = 200)
#learning_rate_function = ReduceLROnPlateau(monitor='val_acc',patience=2,epsilon=0.00001,verbose=1,factor=0.2)
train_history2 = model.fit_generator(train_generator, steps_per_epoch=1500,epochs=25,verbose=1,validation_data=(x_val,y_val))

Epoch 1/25
1500/1500 [==============================] - 129s 86ms/step - loss: 1.7760 - acc: 0.2645 - val_loss: 1.7223 - val_acc: 0.3013
Epoch 2/25
1500/1500 [==============================] - 121s 81ms/step - loss: 1.6912 - acc: 0.3171 - val_loss: 1.6134 - val_acc: 0.3631
Epoch 3/25
1500/1500 [==============================] - 121s 80ms/step - loss: 1.6261 - acc: 0.3566 - val_loss: 1.5877 - val_acc: 0.3743
Epoch 4/25
1500/1500 [==============================] - 121s 81ms/step - loss: 1.5832 - acc: 0.3797 - val_loss: 1.5873 - val_acc: 0.3831
Epoch 5/25
1500/1500 [==============================] - 121s 81ms/step - loss: 1.5541 - acc: 0.3942 - val_loss: 1.4814 - val_acc: 0.4260
Epoch 6/25
1500/1500 [==============================] - 122s 81ms/step - loss: 1.5339 - acc: 0.4032 - val_loss: 1.4757 - val_acc: 0.4314
Epoch 7/25
1500/1500 [==============================] - 121s 81ms/step - loss: 1.5159 - acc: 0.4119 - val_loss: 1.5157 - val_acc: 0.4270
Epoch 8/25
1500/1500 [===================

In [8]:
model_no = MobileNetv2((48, 48, 1), 7, 1.0)
print(model_no.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 48, 48, 1)         0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 24, 24, 32)        320       
_________________________________________________________________
batch_normalization_15 (Batc (None, 24, 24, 32)        128       
_________________________________________________________________
leaky_re_lu_11 (LeakyReLU)   (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 24, 24, 32)        1056      
_________________________________________________________________
batch_normalization_16 (Batc (None, 24, 24, 32)        128       
_________________________________________________________________
leaky_re_lu_12 (LeakyReLU)   (None, 24, 24, 32)        0         
__________

In [76]:
model_tmp = model

In [35]:
epochs = 100
batch_size = 2000
model_no.compile(loss='categorical_crossentropy',
              optimizer= 'Adam',
              metrics=['accuracy'])
model_no.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
        validation_data=(x_val,y_val))
        #callbacks=[learning_rate_function])
#score = MODEL.model.evaluate(x_test, y_test, verbose=0)
#print('Test loss:', score[0])
#print('Test accuracy:', score[1])
#model_no.save('MobileNet.h5')

Train on 22967 samples, validate on 5742 samples
Epoch 1/100
22967/22967 [==============================] - 18s 774us/step - loss: 1.9753 - acc: 0.1806 - val_loss: 1.8669 - val_acc: 0.2461
Epoch 2/100
22967/22967 [==============================] - 7s 322us/step - loss: 1.8165 - acc: 0.2635 - val_loss: 1.7671 - val_acc: 0.2886
Epoch 3/100
22967/22967 [==============================] - 7s 318us/step - loss: 1.7452 - acc: 0.3000 - val_loss: 1.7489 - val_acc: 0.3042
Epoch 4/100
22967/22967 [==============================] - 7s 316us/step - loss: 1.6954 - acc: 0.3232 - val_loss: 1.7535 - val_acc: 0.3116
Epoch 5/100
22967/22967 [==============================] - 7s 320us/step - loss: 1.6489 - acc: 0.3472 - val_loss: 1.7090 - val_acc: 0.3276
Epoch 6/100
22967/22967 [==============================] - 7s 320us/step - loss: 1.6115 - acc: 0.3620 - val_loss: 1.6926 - val_acc: 0.3400
Epoch 7/100
22967/22967 [==============================] - 7s 320us/step - loss: 1.5731 - acc: 0.3853 - val_loss: 1.

KeyboardInterrupt: 

In [2]:
import csv
n = 7178
test_x = []
for i in range(n):
    test_x.append([])

count = 0
with open('test.csv', newline='') as csvfile:
    rows = csv.reader(csvfile)
    for row in rows:
        if(row[0] != "id"):
            test_x[count].append(row[1])
            count += 1
x_test = []
for i in range(len(test_x)):
    x_test.append(test_x[i][0].split(' '))

for i in range(len(test_x)):
    for j in range(len(x_test[0])):
        x_test[i][j] = float(x_test[i][j])
x_test = np.array(x_test)
x_test = x_test.reshape(7178,48,48,1)
x_test = x_test/255

In [3]:
hw3_model = load_model("hw3_model.h5")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [4]:
hw3_predict = hw3_model.predict_classes(x_test)

In [6]:
hw3_predict.shape

(7178,)

In [18]:
hw3_y = np_utils.to_categorical(hw3_predict)

In [101]:
epochs = 450
batch_size = 3000
#model_no.compile(loss='categorical_crossentropy',
#              optimizer= 'Adam',
#              metrics=['accuracy'])
model_tmp.fit(x_test, hw3_y,
          batch_size=batch_size,
          epochs=epochs)
        #callbacks=[learning_rate_function])
#score = MODEL.model.evaluate(x_test, y_test, verbose=0)
#print('Test loss:', score[0])
#print('Test accuracy:', score[1])
#model_no.save('MobileNet.h5')

Epoch 1/450
7178/7178 [==============================] - 1s 175us/step - loss: 0.1389 - acc: 0.9550
Epoch 2/450
7178/7178 [==============================] - 1s 152us/step - loss: 0.1414 - acc: 0.9558
Epoch 3/450
7178/7178 [==============================] - 1s 152us/step - loss: 0.1353 - acc: 0.9576
Epoch 4/450
7178/7178 [==============================] - 1s 154us/step - loss: 0.1390 - acc: 0.9560
Epoch 5/450
7178/7178 [==============================] - 1s 153us/step - loss: 0.1380 - acc: 0.9561
Epoch 6/450
7178/7178 [==============================] - 1s 154us/step - loss: 0.1405 - acc: 0.9557
Epoch 7/450
7178/7178 [==============================] - 1s 157us/step - loss: 0.1337 - acc: 0.9568
Epoch 8/450
7178/7178 [==============================] - 1s 157us/step - loss: 0.1362 - acc: 0.9550
Epoch 9/450
7178/7178 [==============================] - 1s 154us/step - loss: 0.1340 - acc: 0.9578
Epoch 10/450
7178/7178 [==============================] - 1s 153us/step - loss: 0.1398 - acc: 0.9565

In [102]:
prediction = model_tmp.predict(x_test)

In [103]:
prediction2 = np.argmax(prediction,axis=1)

In [104]:
prediction2

array([3, 4, 3, ..., 3, 0, 2])

In [105]:
model_tmp.save("strong.h5")
model_tmp.save_weights("weight_strong.h5")

In [106]:
with open("submission_strong.csv", 'w', newline='') as csvfile:
    csvfile.write('id,label\n')
    for i, v in enumerate(prediction2):
        csvfile.write('%d,%d\n' %(i, prediction2[i]))